In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import sys
print(sys.path)
FOLDERNAME = 'finalproject'
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

%cd /content/drive/My\Drive/$FOLDERNAME

print(sys.path)
%load_ext autoreload
%autoreload 2

Mounted at /content/drive
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython']
/content/drive/MyDrive/finalproject
['', '/content', '/env/python', '/usr/lib/python37.zip', '/usr/lib/python3.7', '/usr/lib/python3.7/lib-dynload', '/usr/local/lib/python3.7/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.7/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/finalproject']


In [2]:
!pip install transformers
!pip install positional_encodings
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
from transformers import BertModel
import json
from torch.utils.data import Dataset, DataLoader
import os
import torch
import torch.nn as nn
from torch import Tensor
import math
from torch.nn import functional as F
import copy
from typing import Optional, List

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import numpy as np
import pandas as pd
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [4]:
from our_transformer import Transformer
from our_detr import build, DETR
from our_matcher import HungarianMatcher
from our_detr import SetCriterion

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# **Loading dataset, Preprocessing**

# **Create dataset for target aspect prediction**

In [21]:
import csv
targets= pd.read_csv("targets.csv")

list = []
for v in targets['sentiment']:
  dicts = {}
  x = v.split(',')
  z = torch.tensor([])
  for i in range(len(x)):
    y = torch.tensor([float(x[i])])
    z = torch.cat((z,y),0)
  dicts['sentiment'] = z
  list.append(dicts)

for l in range(len(targets['target_enc'])):
  x = targets['target_enc'][l].split(',')
  z = torch.tensor([])
  for i in range(len(x)):
    y = torch.tensor([int(x[i])])
    z = torch.cat((z,y),0) 
  list[l]['target'] = z.long()

for l in range(len(targets['aspect_enc'])):
  x = targets['aspect_enc'][l].split(',')
  z = torch.tensor([])
  for i in range(len(x)):
    y = torch.tensor([int(x[i])])
    z = torch.cat((z,y),0) 
  list[l]['aspect'] = z.long()

In [6]:
print(targets['sentiment'])

0      -0.374
1       -0.24
2      -0.161
3       0.137
4       0.704
        ...  
431    -0.807
432     0.339
433     0.439
434     0.435
435     0.259
Name: sentiment, Length: 436, dtype: object


In [32]:
print(list)

[{'sentiment': tensor([-0.3740]), 'target': tensor([0]), 'aspect': tensor([0])}, {'sentiment': tensor([-0.2400]), 'target': tensor([1]), 'aspect': tensor([0])}, {'sentiment': tensor([-0.1610]), 'target': tensor([2]), 'aspect': tensor([0])}, {'sentiment': tensor([0.1370]), 'target': tensor([3]), 'aspect': tensor([0])}, {'sentiment': tensor([0.7040]), 'target': tensor([4]), 'aspect': tensor([0])}, {'sentiment': tensor([0.4460]), 'target': tensor([5]), 'aspect': tensor([1])}, {'sentiment': tensor([0.1360, 0.1920]), 'target': tensor([6, 7]), 'aspect': tensor([0, 0])}, {'sentiment': tensor([0.2130]), 'target': tensor([8]), 'aspect': tensor([0])}, {'sentiment': tensor([0.3890]), 'target': tensor([7]), 'aspect': tensor([1])}, {'sentiment': tensor([0.6500]), 'target': tensor([9]), 'aspect': tensor([0])}, {'sentiment': tensor([0.1950]), 'target': tensor([10]), 'aspect': tensor([0])}, {'sentiment': tensor([0.1190]), 'target': tensor([11]), 'aspect': tensor([0])}, {'sentiment': tensor([0.2420, 0.

# Get Bert and put preprocessed data in




In [7]:
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
for param in model.parameters():
    param.requires_grad = False

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# INPUT TEXT -> WORD EMBEDDING 


In [8]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []
# get sentences from the preprocessed data.
sentences = targets["sentence"]
# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 32,           # Pad & truncate all sentences.
                        padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences)
print('Token IDs:', input_ids)

Original:  0      Royal Mail chairman Donald Brydon set to step ...
1      Stakes High for AstraZeneca Heart Drug Facing ...
2      UPDATE 1-Dairy Crest loses a third of Morrison...
3      Insight hires Aviva's David Hillier for multi-...
4      Primark racks up a happy Christmas after stron...
                             ...                        
431    Aviva, M&G suspend property funds as investors...
432    UK housing market steadies after Brexit dip, P...
433    BRIEF-Aviva aims to increase dividend pay-out ...
434       Builder Persimmon hails 6% rise in house sales
435    EasyJet attracts more passengers in June but s...
Name: sentence, Length: 436, dtype: object
Token IDs: tensor([[  101,  2548,  5653,  ...,     0,     0,     0],
        [  101,  7533,  2152,  ...,     0,     0,     0],
        [  101, 10651,  1015,  ...,     0,     0,     0],
        ...,
        [  101,  4766,  1011,  ...,     0,     0,     0],
        [  101, 12508,  2566,  ...,     0,     0,     0],
     

In [ ]:
criterion= build(1)
criterion.to(device)

SetCriterion(
  (matcher): HungarianMatcher()
)

# **Get positional encoding to put into decoder**

In [9]:
from positional_encodings import PositionalEncoding1D
n = np.zeros(shape=(436, 32, 768))
x = torch.tensor(n, dtype=torch.float32)
print(x.dtype)
p_enc_1d_model = PositionalEncoding1D(768)
p_enc = p_enc_1d_model(x)
print(p_enc.dtype)

torch.float32
torch.float32


# ***Making FULL transformer model with Bert & decoder ***

# **Initialize Transformer**

In [10]:
my_model = Transformer(d_model=768, nhead=8, num_decoder_layers=6, dim_feedforward=2048, dropout=0.1,
                activation="relu", normalize_before=False,
                return_intermediate_dec=False)

# **Initialize DETR with my model**

# **DETR 만들기**

In [23]:
my_detr = DETR(my_model, p_enc,4, 226, 32)
x = my_detr.forward(input_ids, attention_masks)


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1933: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


In [61]:
my_matcher = HungarianMatcher(1, 100, 1)
weight_dict = {'loss_target': 1, 'loss_aspect': 100, 'loss_sentiment' : 10}
losses = ['target', 'aspect', 'sentiment']

In [62]:
my_loss = SetCriterion(226,4, my_matcher, weight_dict, 0.1, 0.1, losses )


In [63]:
y = my_loss.forward(x,list)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([498])) that is different to the input size (torch.Size([13952, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/content/drive/My Drive/finalproject/our_detr.py:151: UserWarning: Using a target size (torch.Size([498])) that is different to the input size (torch.Size([498, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss_sentiment = F.mse_loss(src_sentiment, target_sentiment, reduction='mean')


In [64]:
print(y)

{'loss_target': tensor(5.8152, grad_fn=<NllLoss2DBackward0>), 'loss_aspect': tensor(2.0080, grad_fn=<NllLoss2DBackward0>), 'loss_sentiment': tensor(1.4646, grad_fn=<MulBackward0>)}


In [ ]:
def build(args):
    # the `num_classes` naming here is somewhat misleading.
    # it indeed corresponds to `max_obj_id + 1`, where max_obj_id
    # is the maximum id for a class in your dataset. For example,
    # COCO has a max_obj_id of 90, so we pass `num_classes` to be 91.
    # As another example, for a dataset that has a single class with id 1,
    # you should pass `num_classes` to be 2 (max_obj_id + 1).
    # For more details on this, check the following discussion
    # https://github.com/facebookresearch/detr/issues/108#issuecomment-650269223

    device = torch.device(args.device)
    transformer = build_transformer(args)
    matcher = build_matcher(args)

    model = DETR(
        transformer,
        Positional_Encoding = p_enc,
        num_aspect = 4,
        num_target = 226,
        num_queries= 32,
        aux_loss=args.aux_loss,
    )
    
    weight_dict = {'loss_target': 1, 'loss_aspect': 1, 'loss_sentiment' : 1}
    losses = ['target', 'aspect', 'sentiment']
    if args.masks:
        losses += ["masks"]
    criterion = SetCriterion(4, 226, matcher, weight_dict, eos_coef1, eos_coef2, losses)
    criterion.to(device)
    postprocessors = {'bbox': PostProcess()}
    if args.masks:
        postprocessors['segm'] = PostProcessSegm()
        if args.dataset_file == "coco_panoptic":
            is_thing_map = {i: i <= 90 for i in range(201)}
            postprocessors["panoptic"] = PostProcessPanoptic(is_thing_map, threshold=0.85)

    return model, criterion, postprocessors

# **Hungarian matcher(미완성)**

In [ ]:
class SetCriterion(nn.Module):
    """ This class computes the loss for DETR.
    The process happens in two steps:
        1) we compute hungarian assignment between ground truth sand the outputs of the model
        2) we supervise each pair of matched ground-truth / prediction (supervise class and box)
    """
    def __init__(self, num_targets, num_aspects, matcher, weight_dict, eos_coef1, eos_coef2, losses):
        """ Create the criterion.
        Parameters:
            num_targets: number of target categories, omitting the special no-object category
            num_aspects: number of aspect categories
            matcher: module able to compute a matching between targets and proposals
            weight_dict: dict containing as key the names of the losses and as values their relative weight.
            eos_coef: relative classification weight applied to the no-object category
            losses: list of all the losses to be applied. See get_loss for list of available losses.
        """
        super().__init__()
        self.num_targets = num_targets
        self.num_aspects = num_aspects
        self.matcher = matcher
        self.weight_dict = weight_dict
        self.eos_coef1 = eos_coef1
        self.eos_coef2 = eos_coef2
        self.losses = losses
        empty_weight1 = torch.ones(self.num_targets + 1)
        empty_weight2 = torch.ones(self.num_aspects + 1)
        empty_weight1[-1] = self.eos_coef1
        empty_weight2[-1] = self.eos_coef2
        self.register_buffer('empty_weight1', empty_weight1)
        self.register_buffer('empty_weight2', empty_weight2)

    def loss_targets(self, outputs, targets, indices, num_boxes, log=True):
        """Classification loss (NLL)
        targets dicts must contain the key "targets" containing a tensor of dim [nb_targets]
        """
        assert 'pred_logits' in outputs
        src_logits = outputs['pred_target']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["target"][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight1)
        losses = {'loss_ce': loss_ce}

        #if log:
            # TODO this should probably be a separate loss, not hacked in this one here
        #    losses['class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses
        
    def loss_aspects(self, outputs, targets, indices, num_boxes, log=True):
        """Classification loss (NLL)
        targets dicts must contain the key "aspects" containing a tensor of dim [nb_targets]
        """
        assert 'pred_aspect' in outputs
        src_logits = outputs['pred_aspect']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["aspect"][J] for t, (_, J) in zip(aspects, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss_ce = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.empty_weight)
        losses = {'loss_ce': loss_ce}

        #if log:
            # TODO this should probably be a separate loss, not hacked in this one here
        #      losses['class_error'] = 100 - accuracy(src_logits[idx], target_classes_o)[0]
        return losses

    def loss_sentiment(self, outputs, targets, indices, num_boxes):
        """Compute the losses related to the bounding boxes, the L1 regression loss and the GIoU loss
           targets dicts must contain the key "boxes" containing a tensor of dim [nb_target_boxes, 4]
           The target boxes are expected in format (center_x, center_y, w, h), normalized by the image size.
        """
        assert 'pred_sentiment' in outputs
        idx = self._get_src_permutation_idx(indices)
        src_sentiment = outputs['pred_sentiment'][idx]
        target_sentiment = torch.cat([t['sentiment'][i] for t, (_, i) in zip(targets, indices)], dim=0)
        loss_sentiment = F.mse_loss(src_sentiment, target_sentiment, reduction='none')

        losses = {}
        losses['sentiment'] = loss_sentiment

        return losses

    def _get_src_permutation_idx(self, indices):
        # permute predictions following indices
        batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    def _get_tgt_permutation_idx(self, indices):
        # permute targets following indices
        batch_idx = torch.cat([torch.full_like(tgt, i) for i, (_, tgt) in enumerate(indices)])
        tgt_idx = torch.cat([tgt for (_, tgt) in indices])
        return batch_idx, tgt_idx

    def get_loss(self, loss, outputs, targets, indices, **kwargs):
        loss_map = {
            'target': self.loss_target,
            'aspect': self.loss_aspect,
            'sentiment': self.loss_sentiment,
        }
        assert loss in loss_map, f'do you really want to compute {loss} loss?'
        return loss_map[loss](outputs, targets, indices, **kwargs)

    def forward(self, outputs, targets):
        """ This performs the loss computation.
        Parameters:
             outputs: dict of tensors, see the output specification of the model for the format
             targets: list of dicts, such that len(targets) == batch_size.
                      The expected keys in each dict depends on the losses applied, see each loss' doc
        """
        outputs_without_aux = {k: v for k, v in outputs.items() if k != 'aux_outputs'}

        # Retrieve the matching between the outputs of the last layer and the targets
        indices = self.matcher(outputs_without_aux, targets)
        
        # Compute all the requested losses
        losses = {}
        for loss in self.losses:
            losses.update(self.get_loss(loss, outputs, targets, indices, 3))

        # In case of auxiliary losses, we repeat this process with the output of each intermediate layer.
        if 'aux_outputs' in outputs:
            for i, aux_outputs in enumerate(outputs['aux_outputs']):
                indices = self.matcher(aux_outputs, targets)
                for loss in self.losses:
                    if loss == 'masks':
                        # Intermediate masks losses are too costly to compute, we ignore them.
                        continue
                    kwargs = {}
                    if loss == 'labels':
                        # Logging is enabled only for the last layer
                        kwargs = {'log': False}
                    l_dict = self.get_loss(loss, aux_outputs, targets, indices, num_boxes, **kwargs)
                    l_dict = {k + f'_{i}': v for k, v in l_dict.items()}
                    losses.update(l_dict)

        return losses



In [ ]:
x['pred_target'].shape
x['pred_aspect'].shape

# **Target data **

In [ ]:
targets